# ARD Take 1
Starting to work on this Nov 12, 2015. Given $N$ samples with $M$ dimensions each (selected from $\mathcal{N}(0,1)$), $N$ target values $Y$ (with Gaussian noise added, $\sigma^2$ chosen), and a set of weight vectors $w$ (from an exponential distribution, $\lambda$ chosen, with a certain proportion of the weights (Zeros) set to 0), peform ARD and attempt to recover the weights. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline

In [ ]:
# Generate random design matrix. 
N = 5000     # sample size
M = 50      # dimensions
Zeros = 0.6 # Proportion of zero weights (0 to 1)
noise = 5  # sigma^2 for Gaussian noise added to Y

M_zero = int(Zeros*M)
X = np.random.normal(0, 1, (N,M))

In [ ]:
# Generate weights
w = np.random.laplace(0, 10, M)
perm = np.random.permutation(M)
w[perm[:M_zero]] = 0

# Plotting
f ,ax = plt.subplots(1,2, figsize=(16,5))
ax[0].plot(w)
ax[1].hist(w)
plt.show()

In [ ]:
# Generate Y
Y_noiseless = np.dot(X,w)
Y = Y_noiseless + np.random.normal(0,noise, len(Y_noiseless))  # Add Gaussian Noise

# Plotting
f ,ax = plt.subplots(1,1, figsize=(16,5))
ax.plot(Y, color= 'blue')
ax.plot(Y_noiseless, color = 'green')
plt.show()

# ARD

In [ ]:
# Initialization
alpha_max = 10
max_iterations = 10

a = np.repeat(1.0, M) # alphas
b = 1.0               # Beta = 1/sig^2 

X1 = X
Y1 = Y

deletions = []
all_errors = [0]

for i in range(max_iterations):
    
    # Sigma = (b*XTX + A)^-1
    Sigma = np.linalg.inv(b*np.dot(X1.T,X1) + np.diag(a))

    # mu = b*Sigma*X.T*Y
    mu = b*np.dot(np.dot(Sigma, X1.T), Y1)
    
    gamma = 1.0 - a*np.diag(Sigma)
    a_new = gamma/mu**2
    
    error = np.sum((Y1 - np.dot(X1, mu))**2)
    b_new = (N - np.sum(gamma))/error
    
    a = a_new
    b = b_new
    
    print("\nIteration: ", i, "   beta = ", b, "   Squared-Error = ", error)  
        
    over = [i for i in range(len(a))if a[i] > alpha_max]
    if over:
        print("Deletions: ", len(over))
        deletions = [over] + deletions
        X1 = np.delete(X1,over,axis=1)
        a = np.delete(a,over)
    elif abs(error - all_errors[-1]) < .01:
        break
    all_errors.append(error)
           

# Recover mu
for i in deletions:
    for j in i:
        a = np.insert(a,j,-1)
        mu = np.insert(mu,j,0)

df = pd.DataFrame(list(zip(a, mu, w, mu - w)), columns = ['alpha', 'mu', 'w', 'error'])
print("\n", df)
print("\nDeletions:", np.sum([len(i) for i in deletions]), "out of", M_zero, "\n", deletions)

In [ ]:
# Plotting old weights + new weights, errors
f ,ax = plt.subplots(1,2, figsize=(16,5))
ax[0].plot(mu, color = 'green', label = r'$\mu$')
ax[0].plot(w, color = 'blue', label = r'$w$')
ax[1].plot(w - mu)

ax[0].legend(loc = "upper left", fontsize = 16)
ax[0].set_title(r'Original and Recovered Weights')
ax[0].set_xlabel(r'Weight #')
ax[0].set_ylabel(r'Weight Value')

ax[1].set_title(r'Original - Recovered Weights')
ax[1].set_xlabel(r'Weight #')
ax[1].set_ylabel(r'Error')

plt.show()